# Tutorial on metasyn-disclosure

In this tutorial, we will show how to use the metasyn disclosure control plugin to enhance the privacy of generative metadata (MetaFrames and exported GMF files). 

This tutorial assumes you are familiar with the basic workflow of `metasyn`,  if you're not, please first check out our [metasyn tutorials](https://metasynth.readthedocs.io/en/latest/usage/interactive_tutorials.html).

## Setup

### Installation
The first step is to install the metasyn-disclosure package, this can be done by uncommenting the following line and running it.

In [48]:
# !pip install git+https://github.com/sodascience/metasyn-disclosure-control.git

### Importing Packages
Then we import the necessary packages.

In [49]:
import polars as pl
from metasyn import MetaFrame, demo_dataframe
from metasyn.distribution import (
    DiscreteUniformDistribution,
    FakerDistribution,
    RegexDistribution,
)

from metasyncontrib.disclosure import DisclosurePrivacy

### Loading the dataset

The first step in creating the metadata is reading and converting your dataset to a polars DataFrame. For this tutorial, we will use the demo DataFrame provided by metasyn.  

In [50]:
df = demo_dataframe('titanic') # preconfigured Polars DataFrame based on the Titanic dataset
df.head()

PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],str
1,"""Braund, Mr. Owen Harris""","""male""",22,0,"""A/5 21171""",7.25,null,"""S""",1937-10-28,15:53:04,2022-08-05 04:43:34,null
2,"""Cumings, Mrs. John Bradley (Fl…","""female""",38,0,"""PC 17599""",71.2833,"""C85""","""C""",null,12:26:00,2022-08-07 01:56:33,null
3,"""Heikkinen, Miss. Laina""","""female""",26,0,"""STON/O2. 3101282""",7.925,null,"""S""",1931-09-24,16:08:25,2022-08-04 20:27:37,null
4,"""Futrelle, Mrs. Jacques Heath (…","""female""",35,0,"""113803""",53.1,"""C123""","""S""",1936-11-30,null,2022-08-07 07:05:55,null
5,"""Allen, Mr. William Henry""","""male""",35,0,"""373450""",8.05,null,"""S""",1918-11-07,10:59:08,2022-08-02 15:13:34,null


## Using Disclosure Control

First, let's look at what happens when we fit a MetaFrame to the data without the disclosure plugin.

### Without Disclosure Control

In [51]:
mf = MetaFrame.fit_dataframe(
    df=df,
)

print(
    f"Original Data vs Generative MetaData (for 'Married since' column):\n"
    f"Range in Source DataFrame: {df['Married since'].min()} - {df['Married since'].max()}\n"
    f"Range in Fitted MetaFrame: {mf['Married since'].distribution.lower} - {mf['Married since'].distribution.upper}"
)

100%|██████████| 13/13 [00:01<00:00, 12.51it/s]

Original Data vs Generative MetaData (for 'Married since' column):
Range in Source DataFrame: 2022-07-15 12:21:15 - 2022-08-15 10:32:05
Range in Fitted MetaFrame: 2022-07-15 12:21:15 - 2022-08-15 10:32:05


As you can see, when comparing the original data to the fitted MetaFrame, we can see that the lower bound of the distribution is equal to the lowest value in the source DataFrame, and the upper bound is equal to the highest value. 

This means that source data can be easily inferred from the generative metadata.


### With Disclosure Control

By using the disclosure plugin, we can prevent this privacy concern. 

We can opt to use the disclosure plugin for the entire MetaFrame, by setting the `privacy` parameter to `DisclosurePrivacy()` when fitting the MetaFrame. 

In [52]:
mf_disclosure = MetaFrame.fit_dataframe(
    df=df,
    privacy=DisclosurePrivacy(),  # Make the entire MetaFrame use disclosure control
)

print(
    f"Original Data vs Generative MetaData (for 'Married since' column):\n"
    f"Range in Source DataFrame: {df['Married since'].min()} - {df['Married since'].max()}\n"
    f"Range in Fitted MetaFrame: {mf_disclosure['Married since'].distribution.lower} - {mf_disclosure['Married since'].distribution.upper}"
)

100%|██████████| 13/13 [00:00<00:00, 27.05it/s]

Original Data vs Generative MetaData (for 'Married since' column):
Range in Source DataFrame: 2022-07-15 12:21:15 - 2022-08-15 10:32:05
Range in Fitted MetaFrame: 2022-07-15 17:12:24 - 2022-08-15 07:56:54


As you can see, the disclosure plugin fixes the privacy concern present in the base metasyn package: distribution bounds are no longer equal to the source data. 

### Manually Specifying Distributions
However, there still is a problem. The disclosure plugin does not implement every type of distribution that the base metasyn package does. This means, that in cases where a distribution can not be found, the plugin will default to a NA distribution (returning only NA values).
We can see this happening if we print the generative metadata, for example, notice how the 'Cabin' column has a NA distribution.

In [53]:
print(mf_disclosure)

# Rows: 891
# Columns: 13

Column 1: "PassengerId"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.normal
	- Provenance: metasyn-disclosure
	- Parameters:
		- mean: 446.0
		- sd: 257.18994277900265
	

Column 2: "Name"
- Variable Type: string
- Data Type: String
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.freetext
	- Provenance: metasyn-disclosure
	- Parameters:
		- locale: EN
		- avg_sentences: 2.4691358024691357
		- avg_words: 4.093153759820426
	

Column 3: "Sex"
- Variable Type: categorical
- Data Type: Categorical(ordering='physical')
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.multinoulli
	- Provenance: metasyn-disclosure
	- Parameters:
		- labels: ['female' 'male']
		- probs: [0.35241302 0.64758698]
	

Column 4: "Age"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.1987
- Distribution:
	- Type: core.normal
	- Provenance: metasyn-disclos

We can fix this by manually specifying the distributions for the variables that the disclosure plugin does not support. For example, we can set the 'Cabin' column to have a regex distribution.

In [54]:
specs = [
    # Specify cabin to have a RegexDistribution
    {"name": "Cabin", "distribution": RegexDistribution(r"[ABCDEF][0-9]{2,3}")},
]

mf_disclosure = MetaFrame.fit_dataframe(
    df=df,
    var_specs=specs, 
    privacy=DisclosurePrivacy(),  
)

print(mf_disclosure)


100%|██████████| 13/13 [00:00<00:00, 31.71it/s]

# Rows: 891
# Columns: 13

Column 1: "PassengerId"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.normal
	- Provenance: metasyn-disclosure
	- Parameters:
		- mean: 446.0
		- sd: 257.18994277900265
	

Column 2: "Name"
- Variable Type: string
- Data Type: String
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.freetext
	- Provenance: metasyn-disclosure
	- Parameters:
		- locale: EN
		- avg_sentences: 2.4691358024691357
		- avg_words: 4.093153759820426
	

Column 3: "Sex"
- Variable Type: categorical
- Data Type: Categorical(ordering='physical')
- Proportion of Missing Values: 0.0000
- Distribution:
	- Type: core.multinoulli
	- Provenance: metasyn-disclosure
	- Parameters:
		- labels: ['female' 'male']
		- probs: [0.35241302 0.64758698]
	

Column 4: "Age"
- Variable Type: discrete
- Data Type: Int64
- Proportion of Missing Values: 0.1987
- Distribution:
	- Type: core.normal
	- Provenance: metasyn-disclos

As you can see, the 'Cabin' column now has a RegexDistribution, and is no longer NA.

## Privacy Control for Individual Columns

Instead of using disclosure control for the entire MetaFrame, we can also use it for individual columns.  This way, other columns (which might not be supported by the disclosure plugin) can still use the base metasyn distributions.

To do so, we have to specify the `privacy` parameter for the variable in the variable specification.

We can do this as follows:

In [55]:
specs_disclosure = [
    # Use disclosure control for the 'Married since' column
    {"name": "Married since", "privacy": DisclosurePrivacy()}
]

Then, when fitting the MetaFrame, we have to specify the `dist_providers` parameter to include both the "builtin" distributions, as well as the diclosure variations. 
We have to include them both because otherwise the distributions not specified in the var_spec to use the disclosure plugin will give an error. 

In [56]:
mf_disclosure_alt = MetaFrame.fit_dataframe(
    df=df,
    var_specs=specs_disclosure,
)

print(
    f"Original Data vs Generative MetaData (for 'Married since' column):\n"
    f"Range in Source DataFrame: {df['Married since'].min()} - {df['Married since'].max()}\n"
    f"Range in Fitted MetaFrame: {mf_disclosure_alt['Married since'].distribution.lower} - {mf_disclosure_alt['Married since'].distribution.upper}"
)

mf_disclosure_alt.synthesize(5)

100%|██████████| 13/13 [00:01<00:00, 11.69it/s]

Original Data vs Generative MetaData (for 'Married since' column):
Range in Source DataFrame: 2022-07-15 12:21:15 - 2022-08-15 10:32:05
Range in Fitted MetaFrame: 2022-07-15 17:12:24 - 2022-08-15 07:56:54


PassengerId,Name,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked,Birthday,Board time,Married since,all_NA
i64,str,cat,i64,i64,str,f64,str,cat,date,time,datetime[μs],null
21,"""Protect through. Near school.""","""female""",null,0,"""9913""",9.605397,null,"""S""",1924-10-18,18:21:29,2022-07-19 09:26:51,null
168,"""Study.""","""male""",43,0,"""RE/B 2965""",94.670432,null,"""C""",1904-06-30,18:12:55,2022-08-05 13:43:10,null
439,"""Offer structure.""","""female""",13,0,"""498552""",15.051964,"""C075C60""","""S""",1936-03-10,15:55:05,2022-08-03 09:50:23,null
178,"""Prepare wear writer.""","""female""",48,0,"""004545""",31.906532,null,"""Q""",1939-03-13,18:13:54,2022-08-02 08:52:46,null
668,"""Media. Cut. Level. Pick.""","""female""",27,0,"""GRDE 92149""",1.672398,null,"""Q""",1907-03-13,18:32:49,2022-08-05 14:53:07,null


As you can see by the bounds, the disclosure plugin has been applied effectively for the 'Married since' column.

## Changing the Privacy Level

By default, the disclosure privacy partition size is set to 11.  This partition size determines how data is grouped for privacy protection. Data is grouped into partitions of (at least) the specified size, and the mean of each group is then used. A larger partition size increases privacy but may reduce data utility, while a smaller partition size may increase utility but decrease privacy.

The partition size can be changed by specifying the `partition_size` parameter in the `DisclosurePrivacy` class. This can be done either for the entire MetaFrame, or for individual columns.

When using the plugin for the entire MetaFrame, the syntax is as follows:

In [57]:
mf = MetaFrame.fit_dataframe(
    df=df,
    privacy=DisclosurePrivacy(partition_size=20),  # Set the partition size to 20
)

100%|██████████| 13/13 [00:00<00:00, 28.82it/s]


When using the plugin for individual columns, the syntax is as follows:

In [58]:
specs = [
    {"name": "Married since", "privacy": DisclosurePrivacy(partition_size=20)},
]

mf = MetaFrame.fit_dataframe(
    df=df,
    var_specs=specs,
)

100%|██████████| 13/13 [00:00<00:00, 13.34it/s]


In [77]:
# Fit metaframes
mf_default = MetaFrame.fit_dataframe(
    df=df,
    privacy=DisclosurePrivacy(),  # Default partition size of 11
)

mf_large = MetaFrame.fit_dataframe(
    df=df,
    privacy=DisclosurePrivacy(partition_size=50),  # Larger partition size
)

# Generate synthetic data
synthetic_data_default = mf_default.synthesize()
synthetic_data_large = mf_large.synthesize()

# Convert the 'Married since' column to integer timestamps for easier analysis
synthetic_data_default = synthetic_data_default.with_columns(synthetic_data_default['Married since'].map_elements(lambda x: x.timestamp(), return_dtype=int).alias('Married since'))
synthetic_data_large = synthetic_data_large.with_columns(synthetic_data_large['Married since'].map_elements(lambda x: x.timestamp(), return_dtype=int).alias('Married since'))

# Compare the mean, median, and standard deviation of the 'Married since' column for the default and larger partition sizes.
mean_default = synthetic_data_default['Married since'].mean()
median_default = synthetic_data_default['Married since'].median()
std_dev_default = synthetic_data_default['Married since'].std()

mean_large = synthetic_data_large['Married since'].mean()
median_large = synthetic_data_large['Married since'].median()
std_dev_large = synthetic_data_large['Married since'].std()

# Print the results
print("Default partition size:")
print(f"Mean: {mean_default}, Median: {median_default}, Standard Deviation: {std_dev_default}\n")

print("Larger partition size:")
print(f"Mean: {mean_large}, Median: {median_large}, Standard Deviation: {std_dev_large}")



100%|██████████| 13/13 [00:00<00:00, 25.91it/s]


Default partition size:
Mean: 1659217605.8550186, Median: 1659185364.0, Standard Deviation: 746168.3544682733

Larger partition size:
Mean: 1659206157.7367089, Median: 1659190818.0, Standard Deviation: 766877.7510182475


# Conclusion
That covers the basics of using the disclosure control plugin. If you have any questions, or encounter an issue, please feel free to reach out to us on our [GitHub page](https://github.com/sodascience/metasyn-disclosure-control)